# Extracción de datos con Selenium

![data_ext](images/data_ext.jpeg)

In [ ]:
# Importo todas las librerias que voy a utilizar en este notebook.

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import pandas as pd
import numpy as np

In [ ]:
# Abro el navegador en la pagina del mundo principal para aceptar las cookies, luego directamente le paso la url
# la url del buscador para que reedirija a este buscador con las cookies aceptadas.

path = '.\elmundo\input\chromedriver'
driver = webdriver.Chrome(path)
driver.get('https://elmundo.es')
time.sleep(random.uniform(2.0,4.0))
boton_condiciones = driver.find_element_by_xpath('//button[@id="didomi-notice-agree-button"]')
boton_condiciones.click()
time.sleep(random.uniform(2.0,4.0))
driver.get('https://ariadna.elmundo.es/buscador/archivo.html')
time.sleep(random.uniform(2.0,4.0))
buscador = driver.find_element_by_xpath('//*[@id="formulario"]/label[1]/input')

# Introduce la palabra a buscar y acepta.
keys = 'musica'
buscador.send_keys(keys)
boton_buscar2 = driver.find_element_by_xpath('//*[@id="formulario"]/label[2]/input')
boton_buscar2.click()

In [ ]:
# Una vez en la página, establecemos un bucle por el que irá iterando por las paginas, cogiendo con cada etiqueta 
# un elemento de cada noticia, y lo vamos guardando en diferentes listas. 
numero = 0
url = 'https://ariadna.elmundo.es/buscador/archivo.html?q=musica&t=1&i='+str(numero)+'1&n=10&fd=0&td=0&w=70&s=1'
heads = []
texts = []
otros_datos = []
rango_paginas = range(100)

for i in rango_paginas:
    driver.get(url)


    cajas_noticias = driver.find_elements_by_xpath('//*[@class="lista_resultados"]/li')
    for i in cajas_noticias:
        heads.append(i.find_element_by_tag_name('h3').text)
    textos = driver.find_elements_by_xpath('//*[@class="lista_resultados"]/li/p')
    for j in textos:
        texts.append(j.text)

    fechas = driver.find_elements_by_xpath('//div[@class="metadata_resultado"]')
    for f in fechas:
        f.find_elements_by_class_name('span')
        otros_datos.append(f.text)
    numero += 1
    url = 'https://ariadna.elmundo.es/buscador/archivo.html?q=musica&t=1&i='+str(numero)+'1&n=10&fd=0&td=0&w=70&s=1'
    time.sleep(random.uniform(2.0,4.0))

In [ ]:
# Filtramos algunos elementos que se nos han colado y seleccionamos los 500 elementos de cada lista para construir
# un dataframe.Nota: Algunos de los elementos de las ultimas paginas se habian seleccionado mal, y por eso cojo las
# primeras 500.
for j in heads:
    if j == 'Cultura':
        heads.remove(j)

        
for i in texts:
    if i == 'Noticias relacionadas':
        texts.remove(i)
    else:
        if len(i) < 51:
            texts.remove(i)

cabeceras = heads2[0:500]
textos = texts[0:500]
otros_datos3 = otros_datos[0:500]

In [ ]:
df_elmundo = pd.DataFrame({
    'Titulares': cabeceras,
    'Texto': textos,
    'Otros_datos': otros_datos3
})

In [ ]:
# Guardo el CSV. Nota: Hay que guardarlo con como latin1, ya que todavia tenemos signos propios del español.
df_elmundo.to_csv('df_elmundo.csv', encoding='latin1')